In [ ]:
""" Initialize, import libraries and load data, clean data """

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

import statsmodels.api as sm

# load market data
files_to_load = [
    'BAA10Y', # yield spread between BAA-rated corporate bonds and 10-year Treasury bonds
    'T10Y3M', # term spread between 10-year Treasury bonds and 3-month Treasury bills
    'DTB4WK', # 4-week Treasury bill rate - used as a proxy for the risk-free rate
    'DTB3' # 3-month Treasury bill rate
]
collected_market_data = []
for filename in files_to_load:
    data = pd.read_csv(f'data/{filename}.csv')
    data['observation_date'] = pd.to_datetime(data['observation_date']) - pd.offsets.Day(1)
    data = data.rename(columns={
        'observation_date': 'Date',
    }).set_index('Date')
    collected_market_data.append(data)

df_market_data = pd.concat(collected_market_data, axis=1)
df_market_data['BAA10Y'] /= 100
df_market_data['T10Y3M'] /= 100
df_market_data['DTB3'] /= 100
df_market_data['DTB4WK'] = np.log(1 + df_market_data['DTB4WK'] / 100) # make percentage, take log, add 1 to avoid log(0)


CAPE = pd.read_excel('data/ie_data.xls', sheet_name='Data', skiprows=7, header=0)[['Date', 'CAPE']]
CAPE['Date'] = pd.to_datetime(CAPE['Date'].astype(str), format='%Y.%m')
CAPE = CAPE.dropna()
CAPE['Date'] = CAPE['Date'] - pd.offsets.Day(1)  # Adjusting the date to the next day
CAPE['CAPE'] = np.log(CAPE['CAPE'])  # Taking the log of CAPE for normalization
CAPE = CAPE.set_index('Date')

df_market_data = df_market_data.join(CAPE, how='left')
df_market_data = df_market_data[~df_market_data.index.duplicated(keep='first')]

# load CSM index data
data_path = 'data/index_data/'

with open(data_path + 'run4-mimicking.pkl', 'rb') as f:
    results = pickle.load(f)

trad_country_returns = results["trad_country_returns"]
csm_returns = results["csm_returns"]

trad_country_returns_excess = []
for country in trad_country_returns.columns:
    working_df = trad_country_returns[country].copy()
    working_df = working_df.reset_index()
    working_df['Date'] = pd.to_datetime(working_df['Date'])
    working_df[country] = np.log(1 + working_df[country])
    working_df = working_df.set_index('Date')
    working_df = working_df.join(df_market_data['DTB4WK'])
    country_excess = working_df[country] - working_df['DTB4WK']
    trad_country_returns_excess.append(country_excess)

trad_country_returns_excess = pd.concat(trad_country_returns_excess, axis=1)
trad_country_returns_excess.columns = trad_country_returns.columns
trad_country_returns_excess

csm_returns_excess = []
for country in csm_returns.columns:
    working_df = csm_returns[country].copy()
    working_df = working_df.reset_index()
    working_df['Date'] = pd.to_datetime(working_df['Date'])
    working_df[country] = np.log(1 + working_df[country])
    working_df = working_df.set_index('Date')
    working_df = working_df.join(df_market_data['DTB4WK'])
    country_excess = working_df[country] - working_df['DTB4WK']
    csm_returns_excess.append(country_excess)

csm_returns_excess = pd.concat(csm_returns_excess, axis=1)
csm_returns_excess.columns = csm_returns.columns
csm_returns_excess

### getting global returns is still work in progress


This section provides mean-variance spanning tests (Huberman and Kandel, 1988). This methodology has previously been applied in the context of international diversification. Mean-variance spanning works by selecting a set of initial asset, commonly called benchmark assets. Next to that, there are test assets. Mean-variance spanning provides an answer to the question of whether the mean-variance efficient frontier of the test assets is the same as that of the benchmark assets.

Huberman and Kandel's (HK) approach is that of a regression:

Y = a + bX + e

Where Y is an N x t matrix of returns of test assets; X is a K x t matrix of returns of benchmark assets; a is 1 x K vector of intercepts coefficients and b a 1 x K vector of coefficients. [look at dimensions; do not yet match!]

The HK test looks at the intercepts - in the case where the two mean-variance frontiers are the same, the intercepts will also be 0. Or put differently, when the intercepts are not statistically significant from zero, we can infer that the mean-variance frontier of the test assets is not significantly different from that of the benchmark assets. This, in turn, implies that there are no diversification benefits from adding the test assets.



First we perform iterative mean spanning tests, where have as N benchmark assets the traditional country indices; and each time, we add a different test asset K to the mix. In these cases, K is always 1. The set of K test assets to draw from is the set of individual CSM country indices.

In [ ]:
spanning_results = {}
for country in csm_returns.columns:
    X = sm.add_constant(csm_returns[country])
    y = csm_returns[country]
    model = sm.OLS(y, X).fit()
    spanning_results[country] = model



for test_asset, model in spanning_results.items():
    # print(model.params)
    # print(model.pvalues)
    print(f"Results for CSM index country: {test_asset}")
    print(model.summary())
    print("\n")



The regression below presents the entire model. This is a multivariate model, meaning the lhs of the equation consists of all the test assets (N x t). In order to facilitate the regression tooling software, the data were transformed so that the dependent variable was one long vector of all the columns {r1, ..., rN} of the original matrix R.

R -> [r1, r2, r3]'

Its dimensions are now (N x t) x 1

On the righthand side, the benchmark asset returns were 'repeated' so as to properly match the dependent variable data and new dimensions. The block of R (N x K) was extended N times. So that the dimensions are now (t x N) x K. Previously the shape was N x K.



In [ ]:
# Combined model:
y_combined = csm_returns.values.flatten()
X_combined = np.tile(
    trad_country_returns.values, 
    (csm_returns.shape[1], 1)
)
X_combined = sm.add_constant(X_combined)

combined_model = sm.OLS(y_combined, X_combined).fit()
print(combined_model.summary())

# f_test_result = combined_model.f_test("const = 0")
# print(f_test_result.summary())

The model shows that the regression coefficient of the intercept is different from zero with statistical significance (p < 0.001). The result that not all coefficients are (near) zero suggests that the benchmark assets cannot span the total return variance of test assets and therefore, that there are diversification benefits of the CSM indices.



# Efficient frontier
We can also visualize the efficient frontiers for portfolios constructed using either country index. We construct the efficient frontier as follows. We find a set of weights w such that for various levels of returns, we minimize the volatility of the portfolio:

min (w'Sw)^(1/2)

s.t. sum(w) = 1
and 0 <= w_i <= 1

where w is an 1 x N vector of weights, for N number of countries and S is the N x N sample covariance matrix. The covariance matrix is constructed by simply taking the entire range of monthly returns for each asset. The returns are therefore in monthly terms, which is a bit odd to interpret but does not change the essence of the frontiers.

We perform this exercise twice, once for the CSM indices and again for the traditional country indices. We can then compare the shape of the two.

In [ ]:
from utils import create_efficient_frontier

csm_mean_returns = csm_returns.mean().values
csm_cov_matrix = csm_returns.cov()

trad_mean_returns = trad_country_returns.mean().values
trad_cov_matrix = trad_country_returns.cov()

# combined_returns = pd.concat([csm_returns, trad_country_returns], axis=1)
# combined_mean_returns = combined_returns.mean().values
# combined_cov_matrix = combined_returns.cov()

ef_csm = create_efficient_frontier(csm_mean_returns, csm_cov_matrix)
ef_trad = create_efficient_frontier(trad_mean_returns, trad_cov_matrix)
# ef_combined = create_efficient_frontier(combined_mean_returns, combined_cov_matrix)


In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(ef_csm.volatility * 100, ef_csm.returns * 100, label='Efficient Frontier csm', color='blue', linestyle='-')
plt.plot(ef_trad.volatility * 100, ef_trad.returns * 100, label='Efficient Frontier trad', color='red', linestyle='-')
# plt.plot(ef_combined.volatility * 100, ef_combined.returns * 100, label='Efficient Frontier combined', color='green', linestyle='--')
plt.xlabel('Portfolio Volatility (Std. Dev.)')
plt.ylabel('Portfolio Return')
plt.title('Efficient Frontier')
plt.legend()
plt.grid(True)
plt.show()



# Monte Carlo simulations
This section explores the diversification properties of the CSM indices versus their traditional country proponents using Monte Carlo simulations. For both index types (csm and trad) we randomly sample countries to construct portfolios. For each simulation run, we vary the number of countries we draw. So we randomly draw 2 countries, 3 countries, 5 countries, etc. Each draw, we calculate the equal-weighted portfolio returns of the sampled countries and calculate annualized (geometric) return, standard deviation of the returns and risk/reward ratio (the ratio of the first two metrics - called the Sharpe ratio in the graph, although it is technically not so). This is repeated 10.000 times. We repeat this procedure for specific numbers of countries (2, 3, 5, 7, 10, 13, 16) and for both index types. We finally proceed to aggregate the results by retaining only specific points from the obtained distribution. For each set of outcomes of 10.000 runs we select only the 2.5 percentile, 97.5 percentile and the median observations. 

The results are shown in the graphs below. The first panel shows only the median results for both index types, in one graph. The second panel also presents the lower (2.5 percentile) and upper (97.5 percentile) ranges of the distribution of the results. They are shown in separate plots for clarity. The third panel presents the results for the risk/reward ratio.

In [ ]:
import simulations

# load data
results_quantiles = pd.read_csv('data/simulations_results_quantiles_n_runs_10000.csv', index_col=0)
simulations.plot_std_dev(results_quantiles, metric="std_dev", n_runs=10000, list_of_quantiles=[0.5], facet_plot=False)

In [ ]:
import simulations

# load data
results_quantiles = pd.read_csv('data/simulations_results_quantiles_n_runs_10000.csv', index_col=0)
simulations.plot_std_dev(results_quantiles, metric="std_dev", n_runs=10000, list_of_quantiles=None, facet_plot=True)

In [ ]:
import simulations

# load data
results_quantiles = pd.read_csv('data/simulations_results_quantiles_n_runs_10000.csv', index_col=0)
simulations.plot_std_dev(results_quantiles, metric="sharpe_ratio", n_runs=10000, list_of_quantiles=None, facet_plot=True)

We can visually interpret these results as follows. Let's first look at the change in standard deviation as the number of countries in portfolio increases. We are not directly interested in absolute levels (these are slightly higher on average for CSM indices), we want to look at the shape of each graph and compare that. So this graph, for the CSM indices, starts at around 6.8% (for 2 countries) and ends at 5.9%. This is a decrease of 14%. For the trad indices, the curve starts at 6% (for 2 countries) and ends at 5.4%. This is a decrease of 10%. Moreover, the curve for the traditional country indices seems to be slightly more concave, meaning it approaches the asymptote faster. 

These results suggests that the diversification benefits of traditional country indices are depleted faster. It takes fewer different countries in the index to obtain a decrease in portfolio risk similar to holding all countries. A flipside of this observation is that one could say that these traditional country indices are more alike. For the CSM indices, it takes more countries to approximate the portfolio standard deviation of the full set of countries. This suggests that CSM indices are more different from one another than traditional country indices are.

The statistics mentioned above in the text are summarized in the table below.

In [ ]:
results_quantiles = pd.read_csv('data/simulations_results_quantiles_n_runs_10000.csv', index_col=0)
csm_median_stddev_start = results_quantiles[(results_quantiles['dataset'] == 'csm') & (results_quantiles.index == 0.5) & (results_quantiles['num_countries'] == 2)]['std_dev']
csm_median_stddev_end = results_quantiles[(results_quantiles['dataset'] == 'csm') & (results_quantiles.index == 0.5) & (results_quantiles['num_countries'] == 16)]['std_dev']

trad_median_stddev_start = results_quantiles[(results_quantiles['dataset'] == 'trad') & (results_quantiles.index == 0.5) & (results_quantiles['num_countries'] == 2)]['std_dev']
trad_median_stddev_end = results_quantiles[(results_quantiles['dataset'] == 'trad') & (results_quantiles.index == 0.5) & (results_quantiles['num_countries'] == 16)]['std_dev']

pd.DataFrame({
    'CSM': 
    {
        'median_stddev_start (N=2)': csm_median_stddev_start.values[0],
        'median_stddev_end (N=16)': csm_median_stddev_end.values[0],
        'relative_change': csm_median_stddev_end.values[0] / csm_median_stddev_start.values[0] - 1
    },
    'Traditional': 
    {
        'median_stddev_start (N=2)': trad_median_stddev_start.values[0],
        'median_stddev_end (N=16)': trad_median_stddev_end.values[0],
        'relative_change': trad_median_stddev_end.values[0] / trad_median_stddev_start.values[0] - 1
    }
})


# Section: return decomposition

In this section we explore more closely the return characteristics of the CSM indices. Given that a CSM index is, by design, more closely linked to its sales footprint to a given country (and therefore by its cashflows), we formulate the expectation that the returns are more closely linked to news about cashflows than discount rates - or at least to cashflow news relating to the target country. This point is not trivial and we will develop that in more detail later. In order to test this hypothesis, we will perform a return decomposition of both the CSM indices and traditional country indices, so that we may compare the results. 

An established procedure to study this phenomenon, is through the present value decomposition of excess returns (Shiller and Campbell, 1988), through a vector autoregressive model (Campbell, 1991). This approach has been applied, in many variations, to several research questions in the literature (/cite).

The methodology has its origins in present value models that show that the present value of an asset will be determined by two main components: (1) it will change with unexpected information/news about the cashflows from the asset. If expected cashflows go down, naturally, the value of the asset will go down as well, and vice versa; (2) it will change with the discount rate. If the discount rate with which the cashflows are valued goes down, future cashflows become more valuable. Therefore, the value of the asset will rise. An important distinction between the two components is that a change in discount rates will have an effect on the asset's value now, but also on the expected future return: as the discount rate goes down, initially the value of the asset will rise, but the future expected return will move in the other direction. For news about cashflows, this phenomenon does not occur. Positive (negative) cashflow news will simply lead to a one-off increase (decrease) in the asset's price.

An important thing to note here is that we are talking about _expectations_. The rationale behind focusing on news (sometimes also called innovations) is that only new information will move prices. The (reasonable) assumption is that everything that is already known should be reflected in today's market prices.

Campbell and Shiller's (1988a) original approach to formalize this notion was into the following seminal equation:

r_{t+1} - E_t r_{t+1} = ...

r_{t+1} - E_t r_{t+1} = N_{CF, t+1} - N_{DR, t+1}

where N_CF denotes news about future cash flows, N_DR denotes news about future discount rates.

This equation can be operationalized using Campbell's (1991) VAR approach. The VAR model is designed such that it first estimates the terms Et r_{T+1} (today's expectation of next period returns) and (E_{t+1} - E_t)\sum(rho x r_{t+i+j}) (news about future discount rates). This gives us two of the three terms of equation 2; and we can calculate the remaining term N_{CF}. 

As in Campbell and Polk (2002), we assume the data generating process comes from the following VAR model:

z_{t+1} = a + X z_{t} + u_{t+1}

where z_{t+1} is an m x 1 state vector, a and X are constant parameters with dimensions of m x 1 respectively m x m; and u_{t+1} an i.i.d. m x 1 vector of shocks. The crux of the VAR model lies in the u_{t+1} vector. They are based on the residuals of the model and account for all variance of the asset's return that is not explained by the system of (lagged) state variables. This is important to note, as it underlines the importance of having the state variables correct. One characteristic of these types of VAR models is that the convention is that the first equation is always the lagged (one-period) return. This means the first element of z_{t+1} is r_{t+1}.

In order to get from aggregate shocks to news on specifically cashflows and/or discount rates, we use the following mapping functions:

N_{CF, t+1) = ... 
N_{DR, t+1) = ...

Where e1 is a vector of length m with 1 as its first element and 0's otherwise. This effectively selects only the first row (return component) to measure news sensitivity. \lambda is defined as \lambda = \rho X(I - \rho X)^-1. 

# State variables
There has been quite some variation in the literature on which state variables have been used (see Driessen en Van Zundert (2017) for an overview). As a starting point, we will use those that are commonly used in the literature. But we have to be cautious with using certain holdings / fundamentally based equity criteria. A state variable that is often used is the price/earnings ratio (of which there exist many variations). These are measured on an index level. For the US, the Shiller PE, which is publically available from Prof. Shiller's website, is commonly used. This US Shiller PE is based on price, dividends, earnings and inflation of the S&P500 index. However, for our research question, that may not be a suitable approach, since it is a very close manifestation of the traditional country index we are comparing the CSM index for the US against. Therefore, we will rely mostly on macro-economic factors of a country.

There still is some value in performing the analysis using P/E ratio's, but we will only do this as a robustness check (and we have to be cautious interpreting the results). We will use two variations. One using the US CAPE ratio. Another using a global price/earnings ratio, derived from a global market index. The downsides of the first were already mentioned. The latter approach, using global P/E's, also comes with a downside, which is that it contains information that is not directly relevant to the specific country. If either traditional country indices or CSM indices are a substantially better proxy for a target country's risk, using global fundamentals may not be the best way to measure that. A benefit of using global fundamentals is that it can easily be applied to any country index.


# Data
In this initial version, we will focus on just the US. For the US, it is relatively easy to obtain appropriate data. Moreover, most of the literature on return decomposition is based on US data, which allows us to easier compare our findings with those in the literature.

Suitable candidates for inclusion in the vector of state variables are:
* Excess stock return - log returns. The period returns where convert to log returns. Then subtracted by the log returns of the 1M t-bill rate. Since we have monthly return data, that is the most appropriate value to calculate excess returns. Downloaded from FRED (US open source data). Actually the 4-week t-bill rate, comes closest to the monthly rate but that was not available.
* 3M t-bill yield - converted to percentages. Downloaded from FRED (US open source data).
* Yield spread (10Y-3M) - converted to percentages. Downloaded from FRED (US open source data).
* Default spread - converted to percentages. Downloaded from FRED (US open source data). Default spread is the difference between the yield on a BAA corporate bond and a 10 year government bond.

Robustness:
* PE (US and global)
    * for the US, the Shiller PE was used from Prof. Shiller's website. It was converted to log values.






In [ ]:
import numpy as np
import pandas as pd

# Calculate risk-free returns
US_trad = trad_country_returns['US']
US_trad = US_trad.reset_index()
US_trad['Date'] = pd.to_datetime(US_trad['Date'])
US_trad['US'] = np.log(1 + US_trad['US'])
US_trad = US_trad.set_index('Date')
US_trad = US_trad.join(df_market_data['DTB4WK'])
US_trad_excess = US_trad['US'] - US_trad['DTB4WK']


US_csm = csm_returns['US']
US_csm = US_csm.reset_index()
US_csm['Date'] = pd.to_datetime(US_csm['Date'])
US_csm['US'] = np.log(1 + US_csm['US'])
US_csm = US_csm.set_index('Date')
US_csm = US_csm.join(df_market_data['DTB4WK'])
US_csm_excess = US_csm['US'] - US_csm['DTB4WK']


df_market_data = df_market_data.join(US_trad_excess.rename('US_trad_excess'), how='left')
df_market_data = df_market_data.join(US_csm_excess.rename('US_csm_excess'), how='left')
df_market_data = df_market_data.dropna()

# remove duplicates after joining
df_market_data = df_market_data[~df_market_data.index.duplicated(keep='first')]


In [ ]:
import var_decomposition

data_VAR_trad = df_market_data[[
    "US_trad_excess",
    "BAA10Y",
    "T10Y3M",
    "DTB3",
    # "CAPE"
]]

N_CF, N_DR, results = var_decomposition.get_var_decomp(data_VAR_trad)

corr_mat = np.corrcoef(N_CF, N_DR)
std_N_CF = np.std(N_CF)
std_N_DR = np.std(N_DR)
np.fill_diagonal(corr_mat, [std_N_CF, std_N_DR])
pd.DataFrame(corr_mat, index=['CF', 'DR'], columns=['CF', 'DR']).round(4)



In [ ]:
import var_decomposition

data_VAR_csm = df_market_data[[
    "US_csm_excess",
    "BAA10Y",
    "T10Y3M",
    "DTB3",
    # "CAPE"
]]


N_CF, N_DR, results = var_decomposition.get_var_decomp(data_VAR_csm)

corr_mat = np.corrcoef(N_CF, N_DR)
std_N_CF = np.std(N_CF)
std_N_DR = np.std(N_DR)
np.fill_diagonal(corr_mat, [std_N_CF, std_N_DR])
pd.DataFrame(corr_mat, index=['CF', 'DR'], columns=['CF', 'DR']).round(4)



The two tables above show the correlation between the cashflow and discount rate news components (off-diagonial) and standard deviations (diagonal). The first table's results are for the traditional US country index; the second table is for the CSM US index. Unlike what we initially hypothesized, the results for the two index types are quite similar. Let's first look at the proportion of the standard deviation that each variable explains. These figures (and their proportion) represent how dominant each news type is in explaining a country's market returns. For the traditional country index, the standard deviations of cashflows news (4.8%) and discount rate news (5%) are almost the same. For the CSM indices, it is 5.3% for cashflow news and 6% for discount rate news. These results suggest that the proportion is quite similar; even though we had hypothesized that the cashflow news component would have been more dominant for the CSM indices. If anything, discount rates news is slightly more important for CSM indices.

In [ ]:
import matplotlib.pyplot as plt

N_CF_trad, N_DR_trad, results_trad = var_decomposition.get_var_decomp(data_VAR_trad)
N_CF_csm, N_DR_csm, results_csm = var_decomposition.get_var_decomp(data_VAR_csm)

df_for_plot = pd.DataFrame({
    'Date': data_VAR_trad.reset_index()['Date'][:-1],
    'N_CF_trad': N_CF_trad,
    'N_DR_trad': N_DR_trad,
    'N_CF_csm': N_CF_csm,
    'N_DR_csm': N_DR_csm,
})

fig, axes = plt.subplots(2, 1, figsize=(16, 6), sharey=True)

facet_info = [
    ('N_CF_trad', 'N_DR_trad', 'Traditional Index'),
    ('N_CF_csm', 'N_DR_csm', 'CSM Index')
]

for ax, (cf_col, dr_col, title) in zip(axes, facet_info):
    ax.plot(df_for_plot['Date'], df_for_plot[cf_col], label='a. Cashflow News', color='tab:blue', linestyle='-')
    ax.plot(df_for_plot['Date'], df_for_plot[dr_col], label='b. Discount Rate News', color='tab:orange', linestyle='--')
    ax.set_title(title)
    # if ax is axes[0]:
    #     ax.set_xlabel('Date')
    ax.grid(True)
    ax.set_ylabel('News Component')
    ax.legend()

plt.suptitle('Cashflow and Discount Rate News: Trad and CSM')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


The figure above plots the timeseries of the shocks mapped to each news type; the results are shown distinctly for the traditional US index and the CSM US index. Visually, the charts look very similar.